In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## Online algorithm for the mean

Univariate

In [ ]:
def update_mean(prev_mean,new_data,n):
    new_mean = prev_mean + 1/float(n)*(new_data-prev_mean)
    return new_mean

In [ ]:
print update_mean(0,1,1)
print update_mean(1,0,2)


In [ ]:
values = np.random.randint(0,9,size=(10000))

In [ ]:
values


In [ ]:
values.mean()

In [ ]:
prev_mean = values[0]
n = 1
for new_data in values[1:]:
    n+=1
    new_mean = update_mean(prev_mean,new_data,n)
    prev_mean = new_mean
    print new_mean 
    

In [ ]:
def rolling_expand(arr):
    assert len(arr.shape) == 1, "Array must be 1-dimensional"
    padded_array = np.concatenate([np.nan*np.empty(shape=arr.shape[0]-1),arr],axis=0)
    return np.lib.stride_tricks.as_strided(padded_array,shape=(arr.shape[0],arr.shape[0]),strides=(8,8),writeable=False)
    

In [ ]:
rolling_means = np.nanmean(rolling_expand(values),axis=0)

## Online covariance

In [ ]:
def update_cov_matrix(cov,xn,xn_mean,xn1_mean):
    return cov + np.matmul((xn-xn_mean)[None,:].transpose(),(xn-xn1_mean)[None,:])

In [ ]:
real_cov = np.array([[5,0.5],[0.5,1]])
real_mean = np.array([0,10])

In [ ]:
real_cov, real_mean

In [ ]:
N = 1000

In [ ]:
x = np.random.multivariate_normal(real_mean,real_cov,size=N)
estimated_s = 0
prev_mean = x[0]
for i, xi in enumerate(x[1:]):
    new_mean = update_mean(prev_mean,xi,i+1)
    estimated_s = update_cov_matrix(estimated_s,xi,new_mean,prev_mean)
    prev_mean = new_mean
    
estimated_s/float(N)

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(x[:,0],x[:,1])

## Draft for outlier detector algo

**Formula for online mean update:**

$ \bar{X}_n = \bar{X}_{n-1} + (X_n - \bar{X}_{n-1}) $

**Formula for online Covariance Matrix Estimator update:**

$ S_n = S_{n-1} + (X_n - \bar{X}_n)(X_n - \bar{X}_{n-1})^t $

$ C_n = \frac{1}{n-1}S_n $

In [ ]:
class MockSelf(object):
    pass
self = MockSelf()

In [ ]:
# Ground truth

real_cov = np.array([[5,0.5],[0.5,1]])
real_mean = np.array([0,10])

In [ ]:
p = 10
tmp_mat = np.random.normal(0,2,size=(p,p))
real_cov = np.matmul(tmp_mat.transpose(),tmp_mat)
real_mean = np.random.normal(0,5,size=p)

In [ ]:
# state
self.mean = None
self.n = 0
self.C = 0

In [ ]:
# inputs
features = np.random.multivariate_normal(real_mean,real_cov,size=1)
feature_names = []

# score function

self.n += 1
feat0 = features[0]

if self.mean is None:
    self.mean = feat0
    assert False, 0
else:
    new_mean = self.mean + 1./float(self.n)*(feat0 - self.mean)
    self.C = self.C + np.matmul((feat0 - self.mean)[None,:].transpose(),(feat0 - new_mean)[None,:])
    self.mean = new_mean
    
mean = self.mean
cov = self.C/float(self.n-1)

cov1 = np.linalg.inv(cov)

# Mahalanobis distance = (X-u)t * C-1 * (X-u)
d2 = np.matmul(feat0 - mean, np.matmul(feat0 - mean, cov1.transpose()).transpose())

print features, d2

## Handling Batches

** Formula for online batch mean update (batch of size $b$): **

$ \bar{X}_{n+b} = \bar{X}_n + \frac{b}{n+b}(\bar{X}_{n,n+b} - \bar{X}_n) $

where $\bar{X}_{n,n+b}$ is the mean of the batch.

**Theorem:**

if $A$ and $A+B$ are invertible and $rank(B) = 1$ then

$ (A + B)^{-1} = A^{-1} - \frac{1}{1+trace(BA^{-1})}A^{-1}BA^{-1} $

In our case is B of rank 1 ? Yes:

$ B = (X_n - \bar{X}_n)*(X_n - \bar{X}_{n-1}) $

and 

$ \bar{X}_n = \bar{X}_{n-1} * \frac{n-1}{n} + X_n * \frac{1}{n} $

$ X_n - \bar{X}_n = X_n * ( 1 - \frac{1}{n} ) - \bar{X}_{n-1} * \frac{n-1}{n} $

$ X_n = \frac{n-1}{n}(X_n - \bar{X}_{n-1}) $

So B is produced from a single vector, its rank can only be one

In [ ]:
self.n = 0 
self.mean = 0
self.C = 0

In [ ]:
# inputs
features = np.random.multivariate_normal(real_mean,real_cov,size=1000)
feature_names = []

nb = features.shape[0]

roll_partial_means = features.cumsum(axis=0)/(np.arange(nb)+1).reshape((nb,1))

coefs = (np.arange(nb)+1.)/(np.arange(nb)+self.n+1.)

new_means = self.mean + coefs.reshape((nb,1))*(roll_partial_means-self.mean)

# lets compute rolling (Xn-mXn)t*(Xn-mXn-1)

new_means_offset = np.empty_like(new_means)
new_means_offset[0] = self.mean
new_means_offset[1:] = new_means[:-1]

B = np.matmul((features - new_means)[:,:,None],(features - new_means_offset)[:,None,:])

all_C = self.C + B.cumsum(axis=0)
all_C_inv = np.zeros_like(B)

all_C = np.roll(all_C,1,axis=0)
all_C[0] = self.C

c_inv = None
EPSILON = 1e-8

for i, b in enumerate(B):
    if c_inv is None:
        if abs(np.linalg.det(all_C[i])) > EPSILON:
            c_inv = np.linalg.inv(all_C[i])
        else:
            continue
    else:
        c_inv = all_C_inv[i-1]
    BC1 = np.matmul(b,c_inv)
    all_C_inv[i] = c_inv - 1./(1.+np.trace(BC1))*np.matmul(c_inv,BC1)

all_C_inv *= np.arange(nb).reshape((nb,1,1)) + self.n

self.C += B.sum(axis=0)

self.mean = new_means[-1]

self.n += nb

feat_diff = features-new_means_offset
outlier_scores = np.matmul(feat_diff[:,None,:],np.matmul(all_C_inv,feat_diff[:,:,None])).reshape(nb)

In [ ]:
plt.plot(outlier_scores)

In [ ]:
import OutlierZscore
reload(OutlierZscore)
from OutlierZscore import OutlierZscore

In [ ]:
detector = OutlierZscore()

In [ ]:
detector.score(features,[])